## 4712 errors when extracting #bedrooms

In [17]:
def extractBd(x):
    val = x['facts and features']
    max_idx = val.find(' bd')
    if max_idx < 0:
        max_idx = len(val)
    s = val[:max_idx]

    # find comma before
    split_idx = s.rfind(',')
    if split_idx < 0:
        split_idx = 0
    else:
        split_idx += 2
    r = s[split_idx:]
    return int(r)

In [18]:
# case I: Studio apartments
extractBd({'facts and features' : 'Studio , 1 ba , -- sqft'})
# or extractBd({'facts and features' : 'Studio , 1 ba , 550 sqft'})

ValueError: invalid literal for int() with base 10: '-- sqft'

In [19]:
# case II: multiple apartments listed
extractBd({'facts and features' : '3 $2,800+'})

ValueError: invalid literal for int() with base 10: '00+'

In [20]:
# case III: invalid data
extractBd({'facts and features' : 'Price/sqft: -- , -- bds , -- ba , -- sqft'})

ValueError: invalid literal for int() with base 10: '--'

Issues:
- forgot about Studio apartments
- ill-formatted data with missing info

## Single error row when extracting zipcode

In [22]:
f = lambda x: '%05d' % int(x['postal_code'])

In [23]:
f(None)

TypeError: 'NoneType' object is not subscriptable

=> Issue: Forgot to check for NULL values!

## 167 errors when extracting bathrooms

In [26]:
def extractBa(x):
    val = x['facts and features']
    max_idx = val.find(' ba')
    if max_idx < 0:
        max_idx = len(val)
    s = val[:max_idx]

    # find comma before
    split_idx = s.rfind(',')
    if split_idx < 0:
        split_idx = 0
    else:
        split_idx += 2
    r = s[split_idx:]
    return int(r)

In [31]:
# Row 92 (OperatorID=100007, ecCode=ValueError):
# ('Townhouse For Rent','7 Conant Rd APT 58','WINCHESTER','MA',1890.00000,'$2,850/mo','2 bds , 2.5 ba , 1,409 sqft','https://www.zillow.com/homedetails/7-Conant-Rd-APT-58-Winchester-MA-01890/56541131_zpid/')
# Row 93 (OperatorID=100007, ecCode=ValueError):
# ('Townhouse For Rent','48 Spruce St','Winchester','MA',1890.00000,'$4,250/mo','4 bds , 3.5 ba , 2,000 sqft','https://www.zillow.com/homedetails/48-Spruce-St-Winchester-MA-01890/165961280_zpid/')
# Row 94 (OperatorID=100007, ecCode=ValueError):
# ('Townhouse For Rent','44 Spruce St # 44','Winchester','MA',1890.00000,'$4,500/mo','4 bds , 2.5 ba , 2,500 sqft','https://www.zillow.com/homedetails/44-Spruce-St-44-Winchester-MA-01890/2102250563_zpid/')
extractBa({'facts and features' : '2 bds , 2.5 ba , 1,409 sqft'})

ValueError: invalid literal for int() with base 10: '2.5'

Issue:
- forgot about half rooms, need to clean them explicitly!

## 170 errors when extracting Sqft

In [25]:
def extractSqft(x):
    val = x['facts and features']
    max_idx = val.find(' sqft')
    if max_idx < 0:
        max_idx = len(val)
    s = val[:max_idx]

    split_idx = s.rfind('ba ,')
    if split_idx < 0:
        split_idx = 0
    else:
        split_idx += 5
    r = s[split_idx:]
    r = r.replace(',', '')
    return int(r)

In [27]:
# example: ('House For Rent','232 Washington St','WINCHESTER','MA',1890.00000,'$1,700/mo','2 bds , 1 ba , -- sqft','https://www.zillow.com/homedetails/232-Washington-St-Winchester-MA-01890/56541255_zpid/')

In [28]:
extractSqft({'facts and features' : '2 bds , 1 ba , -- sqft'})

ValueError: invalid literal for int() with base 10: '--'

   +------------+------+
   | ValueError | 4712 |
   +------------+------+
|- filter
|- withColumn
|- filter
|- withColumn
   +-----------+---+
   | TypeError | 1 |
   +-----------+---+
|- mapColumn
|- withColumn
   +------------+---+
   | ValueError | 3 |
   +------------+---+
|- withColumn
   +------------+-----+
   | ValueError | 222 |
   +------------+-----+

In [3]:
# number of exception rows
num_except_rows = 4712 + 1 + 3 + 222
total_rows = 38570 # -1 for header

In [4]:
num_except_rows / total_rows * 100

12.80269639616282